<a href="https://colab.research.google.com/github/kridtapon/Adaptive-Price-Crossover/blob/main/Adaptive_Price_Crossover.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install vectorbt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.6/527.6 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.7/295.7 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 29.6 MB/s eta 0:00:00


In [4]:
import numpy as np
import pandas as pd
import yfinance as yf
import vectorbt as vbt

# Define function to calculate typical price

def typical_price(df):
    return (df['Open'] + df['High'] + df['Low'] + df['Close']) / 4

# Function to calculate moving averages
def moving_average_crossover(df, short_window=50, long_window=200):
    df['TP'] = typical_price(df)  # Calculate typical price
    df['Short_MA'] = df['TP'].rolling(window=short_window).mean()
    df['Long_MA'] = df['TP'].rolling(window=long_window).mean()

    df['Entry'] = df['Short_MA'] > df['Long_MA']  # Buy when short MA crosses above long MA
    df['Exit'] = df['Short_MA'] < df['Long_MA']  # Sell when short MA crosses below long MA
    return df

# Define the stock symbol and time period
symbol = 'BTC-USD'  # Example stock symbol
start_date = '2019-01-01'
end_date = '2025-01-01'

# Download the data
df = yf.download(symbol, start=start_date, end=end_date)
df.columns = ['Close', 'High', 'Low', 'Open', 'Volume']
df.ffill(inplace=True)

# Apply moving average crossover strategy
df = moving_average_crossover(df,1,44)

# Filter data for backtesting period
df = df[(df.index.year >= 2020) & (df.index.year <= 2025)]

# Backtest using vectorbt
portfolio = vbt.Portfolio.from_signals(
    close=df['Close'],
    entries=df['Entry'],
    exits=df['Exit'],
    init_cash=100_000,
    fees=0.001
)

# Display performance metrics
print(portfolio.stats())

# Plot equity curve
portfolio.plot().show()


[*********************100%***********************]  1 of 1 completed


Start                               2020-01-01 00:00:00
End                                 2024-12-31 00:00:00
Period                               1827 days 00:00:00
Start Value                                    100000.0
End Value                                1781471.372996
Total Return [%]                            1681.471373
Benchmark Return [%]                        1197.596406
Max Gross Exposure [%]                            100.0
Total Fees Paid                            61453.198914
Max Drawdown [%]                              56.258416
Max Drawdown Duration                 756 days 00:00:00
Total Trades                                         41
Total Closed Trades                                  41
Total Open Trades                                     0
Open Trade PnL                                      0.0
Win Rate [%]                                  36.585366
Best Trade [%]                               366.732229
Worst Trade [%]                              -11